In [5]:
### Import  librabies
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import DirectoryLoader,PyMuPDFLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import pypdf
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore  



In [6]:
Pine_cone_api_keys="582f464e-7c64-472e-bd99-580598a8ee13"
Pine_cone_api_env="gcp-starter"

In [7]:
# Extract data from pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


extracted_data=load_pdf('data/')

In [8]:
#create text chucks 

def text_split(extracted_data):

    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks

text_chunks=text_split(extracted_data)
print("Length of my chunks:",len(text_chunks))

Length of my chunks: 7020


In [9]:
# Download embedding model

def downloading_embeddingmodel():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings=downloading_embeddingmodel()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [34]:
print("Embedding vector:",embeddings.embed_query("what is acne"))

print("Length of the dimensions",len(embeddings.embed_query("what is acne")))

Embedding vector: [-0.12299400568008423, 0.02277771383523941, -0.03922954574227333, 0.07726240903139114, 0.006203377619385719, 0.028091086074709892, 0.10457874834537506, 0.06359616667032242, -0.04367317259311676, 0.013952089473605156, 0.04262267425656319, -0.025100111961364746, 0.038625605404376984, -0.029103247448801994, -0.02588428184390068, 0.03549799695611, -0.057027384638786316, -0.011699439957737923, 0.0066604395397007465, -0.07993222773075104, -0.06958498805761337, 0.027941683307290077, -0.06689076870679855, -0.13075681030750275, 0.028244733810424805, -0.07138904929161072, 0.021761558949947357, -0.07683862745761871, -0.04244202747941017, -0.006180329713970423, -0.00292257615365088, 0.00979266595095396, -0.05714109539985657, 0.0409608855843544, -0.034957583993673325, 0.002110950415953994, -0.0013601570390164852, -0.0036554757971316576, 0.057528093457221985, 0.0076390416361391544, -0.06129685044288635, 0.011027505621314049, -0.03335292264819145, 0.0036880583502352238, 0.1016863957

In [12]:
# #Initializing the pinecore

# pc = Pinecone(api_key=Pine_cone_api_keys)
# Indexname="medicalchatbot"

# # pinecore converts the text embeddings to vectors and it store it
# docsearch=pc.from_texts([t.page_content for t in text_chunks],embeddings,index_name=Indexname)

Initialize the pinecone database using api keys. and call existing pineconeindex

In [37]:
pc=Pinecone(api_key=Pine_cone_api_keys)
Indexname="medicalchatbot"
index=pc.Index(Indexname)
index.describe_index_stats()

Store the embeddings to pinecone database

In [17]:
# index_name=Indexname
# index = pc.Index(index_name)  
# for i, t in zip(range(len(text_chunks)), text_chunks):
#    query_result = embeddings.embed_query(t.page_content)
#    index.upsert(
#    vectors=[
#         {
#             "id": str(i),  # Convert i to a string
#             "values": query_result, 
#             "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
#         }
#     ],
#     namespace="real" 
# )
# index.describe_index_stats() 

In [18]:
# Prompt Template
promptTemplate='''

Use the following context  of information to answer the user's question.
if you don't know the answer, just say that you don't know, dont try to make up an answer.

Context: {context}
Question: {question}

Only provide helpful answer below and nothing else.
Helpful answer:
'''

In [47]:
llm=CTransformers(model="Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
              model_type="llama",
              config={'max_new_tokens':512,
                      'temperature':0.1})


In [1]:

text_field = "text"  
vectorstore = PineconeVectorStore(  
    index, embeddings  
)  


In [52]:
query = "Abdominal wall defects are effectively treated with surgical repair"  
vectorstore.similarity_search(  
    query,  # our search query  
    k=3  # return 3 most relevant docs  
)  


[]

In [51]:
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
# qa.run(query) 

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _aget_relevant_documents, _get_relevant_documents (type=type_error)

In [50]:
from langchain_pinecone import PineconeVectorStore  
text_field = "text"  
vectorstore = PineconeVectorStore(  
    index, embeddings, text_field  
)  
